<a href="https://colab.research.google.com/github/vianelk/ax-metrics/blob/main/ax_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello")

hello
